# Dask jobqueue worker specification
For setting up Dask jobqueue workers (especially on single-tenant node systems) the full node resources, i.e. available CPUs and free memory have to be known. Here, also a brief description of Dask memory management and how threshold values for Dask jobqueue workers on diskless nodes might be set up is given.

## Node cores

In [1]:
srun --account=esmtst --time=00:05:00 --partition=batch lscpu

srun: job 3029293 queued and waiting for resources
srun: job 3029293 has been allocated resources
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
Address sizes:       46 bits physical, 48 bits virtual
CPU(s):              96
On-line CPU(s) list: 0-95
Thread(s) per core:  2
Core(s) per socket:  24
Socket(s):           2
NUMA node(s):        2
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8168 CPU @ 2.70GHz
Stepping:            4
CPU MHz:             3347.417
CPU max MHz:         3700.0000
CPU min MHz:         1200.0000
BogoMIPS:            5400.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-23,48-71
NUMA node1 CPU(s):   24-47,72-95
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

## Node memory

In [2]:
srun --account=esmtst --time=00:05:00 --partition=batch top -b -n 1 | head -n 5

srun: job 3029296 queued and waiting for resources
srun: job 3029296 has been allocated resources
top - 11:11:10 up 8 days, 17:35,  0 users,  load average: 0.10, 1.72, 7.03
Tasks: 901 total,   1 running, 900 sleeping,   0 stopped,   0 zombie
%Cpu(s):  0.0 us,  0.2 sy,  0.0 ni, 99.8 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
MiB Mem :  94806.0 total,  85146.4 free,   5790.7 used,   3868.9 buff/cache
MiB Swap:      0.0 total,      0.0 free,      0.0 used.  84920.0 avail Mem 


## Worker specification
Based on the outputs above we'll specify Dask jobqueue workers with `cores=96` and `memory=85146MiB`. For SLURM clusters the single Dask workers will automatically inherit memory limits from the specification in the job script header. It is not strictly necessary to provide a "security margin" for the memory already in the job script, as Dask itself is designed to take care of staying below a certain memory limit and will pause and/or even kill a worker process that gets close to / or exceeds the specified memory limit.

The default memory thresholds are:

In [1]:
python -c "import dask.distributed; print(dask.config.get('distributed.worker.memory'))"

{'target': 0.6, 'spill': 0.7, 'pause': 0.8, 'terminate': 0.95}


The `pause` option is the value at which a worker stops accepting new tasks, but will continue with calculations (which might still increase memory utilization). The `terminate` option specifies at which memory usage to kill a Dask worker to prevent e.g. crashing of the host system. For more details on memory options consider the Dask distributed documentation available [here](https://distributed.dask.org/en/latest/worker.html#memory-management).

On the diskless nodes operated at JSC we might decrease the default Dask worker termination memory threshold and use only 90% of the memory indicated as "free" above. Unfortunately, these values cannot be set via the Dask jobqueue cluster specification directly, and we'll make use of [configuring Dask](https://docs.dask.org/en/latest/configuration.html) using its system environment variable approach.

In [2]:
unset $(compgen -v | grep DASK_)
export DASK_DISTRIBUTED__WORKER__MEMORY__TERMINATE=0.90
python -c "import dask.distributed; print(dask.config.get('distributed.worker.memory'))"

{'terminate': 0.9, 'target': 0.6, 'spill': 0.7, 'pause': 0.8}


### Disable disk spilling
Dask workers will, per default, spill their least recently used task data to disk (see [here](https://distributed.dask.org/en/latest/worker.html#spill-data-to-disk) for details). On the diskless nodes operated at JSC there is no temporary disk storage, and using the distributed storage as `local-directory` will cause Dask worker performance to degrade noticably. To completely [prevent disk spilling](https://docs.dask.org/en/latest/setup/hpc.html) the `target` and `spill` memory threshold options should be set as follows:

In [3]:
unset $(compgen -v | grep DASK_)
export DASK_DISTRIBUTED__WORKER__MEMORY__TARGET=False
export DASK_DISTRIBUTED__WORKER__MEMORY__SPILL=False
python -c "import dask.distributed; print(dask.config.get('distributed.worker.memory'))"

{'spill': False, 'target': False, 'pause': 0.8, 'terminate': 0.95}
